In [1]:
import numpy as np
import open3d as o3d
from tqdm import tqdm
from growing import RegionGrowing1 as reg1
from collections import deque

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
datas = o3d.io.read_point_cloud('Point Cloud Data\Corner.ply')
xyz = np.asarray(datas.points)

In [5]:
def SVD(points):
    # 二维，三维均适用
    # 二维直线，三维平面
    pts = points.copy()
    # 奇异值分解
    c = np.mean(pts, axis=0)
    A = pts - c # shift the points
    A = A.T #3*n
    u, s, vh = np.linalg.svd(A, full_matrices=False, compute_uv=True) # A=u*s*vh
    normal = u[:,-1]

    # 法向量归一化
    nlen = np.sqrt(np.dot(normal,normal))
    normal = normal / nlen
    # normal 是主方向的方向向量 与PCA最小特征值对应的特征向量是垂直关系
    # u 每一列是一个方向
    # s 是对应的特征值
    # c >>> 点的中心
    # normal >>> 拟合的方向向量
    return u,s,c,normal

def estimate_parameters(pts):
        # 最小二乘法估算平面模型
        # 只有三个点时，可以直接计算

        _,_,c,n = SVD(pts)

        params = np.hstack((c.reshape(1,-1),n.reshape(1,-1)))[0,:]
        return params

In [6]:
xyz = xyz[np.lexsort(xyz.T)]
Nlrp = 1000
LRP = np.average(xyz[:Nlrp,2])
Thseeds = 0.1
seed = xyz[xyz[:,2] < Thseeds + LRP ]
seed_cp = seed
for i in range(10):    
    params = estimate_parameters(pts=seed_cp)
    h = (xyz[:,0]-params[0]) * params[3] + (xyz[:,1]-params[1])* params[4]+((xyz[:,2]-params[2])* params[5])/(np.sqrt(params[3]**2+params[4]**2+params[5]**2))
    seed_cp = xyz[h < 0.2]

In [7]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(seed_cp)
# o3d.visualization.draw_geometries([pcd])
pcd.estimate_normals()

In [8]:
cure = []
neighbour_number = 30

In [9]:
def first_seed(pcd,cure):
        num_of_pts = len(pcd.points)         # 点云点的个数
        pcd.estimate_covariances(o3d.geometry.KDTreeSearchParamKNN(neighbour_number))
        cov_mat = pcd.covariances            # 获取每个点的协方差矩阵
        cure = np.zeros(num_of_pts)          # 初始化存储每个点曲率的容器
        point_curvature_index = np.zeros((num_of_pts, 2))
        # 计算每个点的曲率
        for i_n in tqdm(range(num_of_pts)):
            eignvalue, _ = np.linalg.eig(cov_mat[i_n])  # SVD分解求特征值
            idx = eignvalue.argsort()[::-1]
            eignvalue = eignvalue[idx]
            cure[i_n] = eignvalue[2] / (eignvalue[0] + eignvalue[1] + eignvalue[2])
        
        cure = np.array(cure)

        return cure

In [10]:
cure = first_seed(pcd,cure)

100%|██████████| 110623/110623 [00:04<00:00, 23754.19it/s]


In [11]:
seed = np.argsort(cure)[0]
paves = np.array([seed])
theta_threshold = 15
cosine_threshold = np.cos(np.deg2rad(theta_threshold))
curvature_threshold=0.035


In [12]:
def find_neighbour_points(cloud,kdtree):
    number = len(pcd.points)
    
    point_neighbours = np.zeros((number, neighbour_number))
    for ik in tqdm(range(number)):
        [_, idx, _] = kdtree.search_knn_vector_3d(pcd.points[ik], neighbour_number)  # K近邻搜索
        point_neighbours[ik, :] = idx
    return point_neighbours 

In [13]:
kdtree = o3d.geometry.KDTreeFlann(pcd)
nebor_all = find_neighbour_points(pcd,kdtree)

100%|██████████| 110623/110623 [00:02<00:00, 51371.86it/s]


In [14]:
seed = np.array([seed])

In [15]:

while len(seed)>0:
    seed_now = seed[0]
    nebor = nebor_all[seed_now]

    nebor = np.asarray(nebor)


    nebor_np = nebor[np.isin(nebor,paves,invert=True)]
    nebor_new = nebor_np[np.isin(nebor_np,seed,invert=True)]

    if len(nebor_new)>0:

        curr_seed_normal = pcd.normals[seed_now]       # 当前种子点的法向量
        seed_nebor_normal = [pcd.normals[int(i)]  for i in nebor_new]     # 种子点邻域点的法向量
        dot_normal = np.fabs(np.dot(seed_nebor_normal, curr_seed_normal))
        nebor_new = nebor_new.astype('int64')
        cure_now= cure[nebor_new]
        a = dot_normal > cosine_threshold
        b = cure_now < curvature_threshold
        c = a&b



        paves_new = nebor_new[c]

        paves = np.append(paves,paves_new)
        seed = np.append(seed,paves_new)



    # for i in nebor_new:
    #     curr_seed_normal = pcd.normals[seed_now]       # 当前种子点的法向量
    #     seed_nebor_normal = pcd.normals[i]      # 种子点邻域点的法向量
    #     dot_normal = np.fabs(np.dot(seed_nebor_normal, curr_seed_normal))

    #     if dot_normal > cosine_threshold:
    #         cure_now = cure[i]
    #         if cure_now < curvature_threshold:
    #             paves = np.append(paves,i)
    #             seed = np.append(seed,i)




    seed = np.delete(seed,[0])


In [16]:
inliers = list(paves)


In [17]:
inlier_cloud = pcd.select_by_index(inliers)
inlier_cloud.paint_uniform_color([1.0, 0, 0])
outlier_cloud = pcd.select_by_index(inliers, invert=True)
outlier_cloud.paint_uniform_color([0,1,0])
o3d.visualization.draw_geometries([inlier_cloud,outlier_cloud])


[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.
[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.
